<a href="https://colab.research.google.com/github/fogg-lab/tissue-model-analysis-tools/blob/main/notebooks/analysis_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tissue Model Analysis Tools Demo

## Description
Runs `fl_tissue_model_analysis_tools` on example data from the [tissue-model-analysis-tools-data](https://github.com/fogg-lab/tissue-model-analysis-tools-data) repository.

## Instructions

### 1) Use a GPU runtime
Change the Colab runtime type to use GPU acceleration, so that the invasion depth and microvessel analysis scripts run faster.
> In the toolbar, go to Runtime -> Change runtime type. Under "Hardware accelerator", select "GPU" and choose a GPU type (any option works).

### 2) Run the setup section

Run the setup cells to set up the Conda environment and the fl_tissue_model_tools package in the Colab runtime. Installing the condacolab package will restart the runtime, and you might get a warning that says "Your session crashed for an unknown reason." You can safely ignore that message.

### 3) Run the demo

We run each analysis using the scripts from the [fl_tissue_model_tools](https://github.com/fogg-lab/tissue-model-analysis-tools) package, via the command line utility (`tmat`) that is installed alongside the package. In this notebook, these commands are prefixed with an exclamation mark (`!tmat ...`). The equivalent command to run in a regular terminal is the same, but without the `!`.

After running the setup cells, you can customize the scripts and their configurations by modifying the files stored in tmat's base directory: /content/fl_tissue_model_tools.

Alternatively, you can run these analyses on your own data. For best results on invasion depth and microvessel formation analysis, you will need to train your own models. To train the invasion depth classifier and the microvessel segmentation model (on a CUDA-capable system), refer to the [training notebooks](https://github.com/fogg-lab/tissue-model-analysis-tools/tree/main/notebooks) in the GitHub repository.

## Setup

### 1. Install conda in the Colab runtime
**This will restart the runtime.**

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### 2. Install packages
**The installation will take around 5-10 minutes.**

In [ ]:
!mamba env update -f https://github.com/fogg-lab/tissue-model-analysis-tools/raw/main/conda_environments/environment_colab.yml
!tmat configure /content/fl_tissue_model_tools

# also remove colab's "sample_data" folder (it is distracting)
!rm -rf /content/sample_data 2> /dev/null || true

### 3. Download example input data
**~2 minutes**

In [ ]:
!git clone https://github.com/fogg-lab/tissue-model-analysis-tools-data.git

## Print `tmat` package usage with the `--help` (`-h`) flag

In [ ]:
!tmat -h

## Z Projection

**Runs this script:** /content/fl_tissue_model_tools/scripts/compute_zproj.py

**Input images:** /content/tissue-model-analysis-tools-data/zprojection_input

In [ ]:
# Run the Z projection utility
!tmat compute_zproj --verbose /content/tissue-model-analysis-tools-data/zprojection_input /content/zproj_demo_out

### Output
Stored in /content/zproj_demo_out

Show the projections side-by-side:

In [ ]:
from matplotlib import pyplot as plt
from skimage import io
from glob import glob
import os

zprojection_paths = glob('/content/zproj_demo_out/*.tif')
zprojections = [io.imread(zp) for zp in zprojection_paths]

plt.figure(figsize=(16, 16))

for index, (zprojection, zp_path) in enumerate(zip(zprojections, zprojection_paths), start=1):
    plt.subplot(1, len(zprojections), index)
    plt.imshow(zprojection, cmap='gray')
    plt.axis('off')
    
    image_name = os.path.splitext(os.path.basename(zp_path))[0]
    plt.title(image_name)

plt.show()

## Cell Coverage Area Analysis

**Runs this script:** /content/fl_tissue_model_tools/scripts/compute_cell_area.py

**Configuration:** /content/fl_tissue_model_tools/config/default_cell_area_computation.json

**Input images:** /content/tissue-model-analysis-tools-data/zprojection_input

In [ ]:
# Run the cell area script
!tmat compute_cell_area --verbose /content/tissue-model-analysis-tools-data/cell_coverage_area_input /content/cellarea_demo_out

### Output

Stored in /content/cellarea_demo_out

*You can also load the .csv output in Excel or any other spreadsheet program.*

In [ ]:
!cat /content/cellarea_demo_out/calculations/cell_area.csv | perl -pe 's/((?<=,)|(?<=^)),/ ,/g;' | column -t -s,

## Invasion Depth Analysis

**Runs this script:** /content/fl_tissue_model_tools/scripts/compute_inv_depth.py

**Configuration:** /content/fl_tissue_model_tools/config/default_invasion_depth_computation.json

**Input images:** /content/tissue-model-analysis-tools-data/invasion_depth_input

**Trained model data:** /content/fl_tissue_model_tools/model_training/best_ensemble

In [ ]:
# Run the invasion depth script
!tmat compute_inv_depth --verbose /content/tissue-model-analysis-tools-data/invasion_depth_input /content/invasion_demo_out

### Output

Full output is stored in /content/invasion_demo_out

*You can also load the .csv output in Excel or any other spreadsheet program.*

In [ ]:
!cat /content/invasion_demo_out/invasion_depth_predictions.csv | perl -pe 's/((?<=,)|(?<=^)),/ ,/g;' | column -t -s,

## Microvessel Formation Analysis

*We are currently working on making this analysis more efficient for both GPU and CPU inference, using techniques like quantization-aware training and model compression.*

**Runs this script:** /content/fl_tissue_model_tools/scripts/compute_branches.py

**Configuration:** /content/fl_tissue_model_tools/config/default_branching_computation.json

**Input images:** /content/tissue-model-analysis-tools-data/branching_input

**Trained model data:** /content/fl_tissue_model_tools/model_training/binary_segmentation

In [ ]:
# Copy `n` images from the full set of demo input images into a smaller directory to run the analysis on
n = 10
!cd /content/tissue-model-analysis-tools-data/branching_input && mkdir -p input_set && find . -maxdepth 1 -type f -iname "*.tif" | head -${n} | xargs -I '{}' cp '{}' input_set/

# Run the branching script with the --save-graphics (-g) flag to save visualizations.
!tmat compute_branches --verbose -g /content/tissue-model-analysis-tools-data/branching_input/input_set /content/branching_demo_out
# Alternatively, you can use the --save-intermediates flag (-i) to also save intermediate visualizations
# !tmat compute_branches --verbose -i /content/...

### Output

Stored in /content/branching_demo_out

*You can also load the .csv output in Excel or any other spreadsheet program.*

#### 1. Show the .csv output

In [ ]:
import pandas as pd

csv_path = '/content/branching_demo_out/branching_analysis.csv'
df = pd.read_csv(csv_path, encoding='utf-16')
print(df.to_string(index=False))

#### 2. Show visualizations for 5 random images

*Samples with no branches are skipped.*

In [ ]:
import os
import random
import pandas as pd
from glob import glob
from skimage import io
from matplotlib import pyplot as plt

# Show visualizations for 5 randomly sampled images. Samples with no branches are skipped

vis_dir = "/content/branching_demo_out/visualizations"
csv_path = "/content/branching_demo_out/branching_analysis.csv"

df = pd.read_csv(csv_path, encoding='utf-16')
filtered_df = df[df['Total # of branches'] > 0]

subdirs = [os.path.join(vis_dir, d) for d in filtered_df['Image'] if os.path.isdir(os.path.join(vis_dir, d))]
selected_subdirs = random.sample(subdirs, 5)

vis_filenames = ["original_image.png", "segmentation_mask.png", "morse_tree.png", "barcode.png"]
subplot_titles = ["Original Image", "Segmentation Mask", "Morse Tree", "Persistent Homology Barcode"]

for subdir in selected_subdirs:
    plt.figure(figsize=(15, 15))
    plt.suptitle(os.path.basename(subdir))

    for index, file_name in enumerate(vis_filenames, start=1):
        file_path = os.path.join(subdir, file_name)
        image = io.imread(file_path)

        plt.subplot(2, 2, index)
        plt.imshow(image, cmap='gray' if index != 4 else 'jet')
        plt.title(subplot_titles[index-1])
        plt.axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()